### Workbook to cycle through zipcodes listed in the SAMHSA datasets and aggregate into counts per zipcode

In [35]:
import pandas as pd

In [36]:
def city_merge_and_sum(city, column, zipcode_dictionary, zipcode_dataset):
    
  merged = pd.merge(zipcode_dictionary, zipcode_dataset, on="Zipcode")
  
  cols=[i for i in merged.columns if i not in ["Zipcode","City","County"]]
  for col in cols:
    merged[col]=pd.to_numeric(merged[col])
    
  return merged[merged['City']==city][column].sum()

# Where...
# city: 'city_name'
# column: 'column_of_interest'
# zipcode_dictionary: dataframe; make sure column for zipcode = 'zipcode'
# zipcode_dataset: dataframe; make sure column for zipcode = 'zipcode'

# and returns:
# the sum of the 'column_of_interest' for a given city


In [37]:
def cities_merge_and_sum(column, zipcode_dictionary, zipcode_dataset):
    merged = pd.merge(zipcode_dictionary, zipcode_dataset, on="Zipcode")
    
    cols=[i for i in merged.columns if i not in ['Zipcode','City','County']]
    for col in cols:
        merged[col]=pd.to_numeric(merged[col])

    City = []
    Value = []
    
    for x in sorted(list(set(zipcode_dictionary['City']))):
        City.append(x)
        Value.append(merged[merged['City']==x][column].sum())
    return City, Value

# Where...
# column: 'column_of_interest'
# zipcode_dictionary: dataframe; make sure column for zipcode = 'zipcode'
# zipcode_dataset: dataframe; make sure column for zipcode = 'zipcode'

# and returns two lists:
# (1) the cities, and
# (2) sum of the 'column_of_interest' for a all cities zipcode dinctionary


In [38]:
# import the massachusettes zipcode dictionary
massachusetts_zipcode_dictionary = pd.read_csv('~/Desktop/new_insights/data_files/final_zipcode_dictionary.csv', dtype = 'str', index_col=False)
massachusetts_zipcode_dictionary = pd.DataFrame(massachusetts_zipcode_dictionary)
massachusetts_zipcode_dictionary.head()

,Zipcode,City,County
0,01001,Agawam,Hampden
1,01002,Amherst,Hampshire
2,01005,Barre,Worcester
3,01007,Belchertown,Hampshire
4,01008,Blandford,Hampden


In [39]:
target_zipcodes = massachusetts_zipcode_dictionary['Zipcode']
type(target_zipcodes)

pandas.core.series.Series

In [40]:
import csv

In [41]:
# open the file in universal line ending mode 
#with open('~/Desktop/new_insights/data_files/opioid_services.csv', 'rU') as infile:
with open('/Users/matt/Desktop/new_insights/data_files/opioid_services.csv', 'rU') as infile:
  # read the file as a dictionary for each row ({header : value})
  reader = csv.DictReader(infile)
  data = {}
  for row in reader:
    for header, value in row.items():
      try:
        data[header].append(value)
      except KeyError:
        data[header] = [value]


/Users/matt/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 'U' mode is deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [42]:
# extract the variables you want
samhsa_certified = data['\ufeffSA_samhsa_certified']
buprenorphrine_physicians = data['buprenorphrine_physicians']
EMHS_crisis_intervention = data['EMHS_crisis_intervention']
SA_transitional_housing = data['SA_transitional_housing']
SA_opioid_detoxification = data['SA_opioid_detoxification']

In [43]:
samhsa_certified_list = list(filter(None, samhsa_certified)) # fastest
buprenorphrine_physicians_list = list(filter(None, buprenorphrine_physicians)) # fastest
EMHS_crisis_intervention_list = list(filter(None, EMHS_crisis_intervention)) # fastest
SA_transitional_housing_list = list(filter(None, SA_transitional_housing)) # fastest
SA_opioid_detoxification_list = list(filter(None, SA_opioid_detoxification)) # fastest

In [44]:
len(SA_opioid_detoxification_list)

77

In [45]:
def zipcode_services_count(zipcode_of_services, zipcode_dictionary):
    zc = []
    services = []
    
    for zipcode in zipcode_dictionary['Zipcode']:
        zc.append(zipcode)
        if zipcode in zipcode_of_services:
            services.append(zipcode_of_services.count(zipcode))
        else:
            services.append(0)
    return zc, services
        

In [46]:
zipcode, samhsa_certified_zips = zipcode_services_count(samhsa_certified_list, 
                                                        massachusetts_zipcode_dictionary)
zipcode, buprenorphrine_physicians_zips = zipcode_services_count(buprenorphrine_physicians_list, 
                                                        massachusetts_zipcode_dictionary)
zipcode, EMHS_crisis_intervention_zips = zipcode_services_count(EMHS_crisis_intervention_list, 
                                                        massachusetts_zipcode_dictionary)
zipcode, SA_transitional_housing_zips = zipcode_services_count(SA_transitional_housing_list, 
                                                        massachusetts_zipcode_dictionary)
zipcode, SA_opioid_detoxification_zips = zipcode_services_count(SA_opioid_detoxification_list, 
                                                        massachusetts_zipcode_dictionary)

In [47]:
services_data = {'Zipcode':zipcode,
        'SAMHSA Certified':samhsa_certified_zips,
        'Buprenorphrine Physicians':buprenorphrine_physicians_zips,
        'EMHS Crisis Intervention':EMHS_crisis_intervention_zips,
        'Transitional Housing':SA_transitional_housing_zips,
        'Opioid Detoxification':SA_opioid_detoxification_zips
       }
massachusetts_zipcode_number_services = pd.DataFrame(services_data)
massachusetts_zipcode_number_services.head()

,Zipcode,SAMHSA Certified,Buprenorphrine Physicians,EMHS Crisis Intervention,Transitional Housing,Opioid Detoxification
0,01001,0,0,0,0,0
1,01002,0,7,0,0,0
2,01005,0,9,0,0,0
3,01007,0,4,0,0,0
4,01008,0,0,0,0,0


In [48]:
### write out MA zipcode services count
massachusetts_zipcode_number_services.to_csv('~/Desktop/new_insights/data_files/massachusetts_zipcode_health_services.csv', index = False)

In [49]:
city_merge_and_sum('Boston', 'Opioid Detoxification',
                   zipcode_dictionary = massachusetts_zipcode_dictionary,
                   zipcode_dataset = number_services)

10

In [50]:
zipcode_dictionary = massachusetts_zipcode_dictionary
zipcode_dataset = massachusetts_zipcode_number_services


city, samhsa_certified = cities_merge_and_sum('SAMHSA Certified',
                     zipcode_dictionary,
                     zipcode_dataset)
city, buprenorphrine_physicians = cities_merge_and_sum('Buprenorphrine Physicians',
                     zipcode_dictionary,
                     zipcode_dataset)
city, EMHS_crisis_intervention = cities_merge_and_sum('EMHS Crisis Intervention',
                     zipcode_dictionary,
                     zipcode_dataset)
city, SA_transitional_housing = cities_merge_and_sum('Transitional Housing',
                     zipcode_dictionary,
                     zipcode_dataset)
city, SA_opioid_detoxification = cities_merge_and_sum('Opioid Detoxification',
                     zipcode_dictionary,
                     zipcode_dataset)

In [51]:
type(SA_opioid_detoxification)

list

In [52]:
services_data = {'City':city,
        'SAMHSA Certified':samhsa_certified,
        'Buprenorphrine Physicians':buprenorphrine_physicians,
        'EMHS Crisis Intervention':EMHS_crisis_intervention,
        'Transitional Housing':SA_transitional_housing,
        'Opioid Detoxification':SA_opioid_detoxification
       }

In [53]:
city_services = pd.DataFrame(services_data)

In [54]:
city_services

,City,SAMHSA Certified,Buprenorphrine Physicians,EMHS Crisis Intervention,Transitional Housing,Opioid Detoxification
0,Abington,0,1,0,0,0
1,Acton,0,0,0,0,0
2,Acushnet,0,0,0,0,0
3,Adams,0,0,0,0,0
4,Agawam,0,0,1,0,0
...,...,...,...,...,...,...
298,Woburn,1,15,0,1,1
299,Worcester,7,126,1,3,6
300,Worthington,0,0,0,0,0
301,Wrentham,0,0,0,0,0


In [55]:
city_services.to_csv('~/Desktop/new_insights/data_files/massachusetts_cities_health_services.csv', index = False)